# Python-Oracle 연동 - 데이터 조회
## 01 준비과정

### 1. 패키지 참조
`cx_oracle`, `sqlalchemy`, `pandas`,`openpyxl`, `xlrd`은 미리 깔려있어야 함

In [2]:

import cx_Oracle as cx  # '0'에 대문자 주의
from sqlalchemy import create_engine
from pandas import DataFrame, read_sql_table

## 02. cx_Oracle을 사용한 연동
데이터베이스 연동 과정은 데이터베이스 접속 <br/>-> SQL 실행 객체(cursor) 생성<br/> -> 결과 처리<br/> -> 데이터베이스 접속 해제의 순서로 진행된다

### 1. 데이터베이스 접속

In [3]:
dbcon = cx.connect("hr",                # 사용자 이름
                    "hr",               # 비밀번호
                    "localhost:1521/xe" # 데이터베이스 서버 주소
)
dbcon

<cx_Oracle.Connection to hr@localhost:1521/xe>

### 2. Cursor 객체 생성

In [4]:
cursor = dbcon.cursor()

### 3. 데이터 조회를 위한 SQL문 수행

#### (1) 튜플을 원소로 갖는 리스트 형태로 조회

In [5]:
sql = "SELECT * from department"
try:
    cursor.execute(sql)
    result = cursor.fetchall()
    print(result)
except Exception as e:
    print("데이터 조회 실패", e)

[(101, '컴퓨터공학과', '1호관'), (102, '멀티미디어학과', '2호관'), (201, '전자공학과', '3호관'), (202, '컴퓨터공학과', '공학관'), (302, '정보통신학과', '6호관')]


#### (2) 딕셔너리를 원소로 갖는 리스트 형태로 조회

cx_Oracle은 별도의 딕셔너리 형태를 제공하지 않는다.
cursor객체의 rowfactory 프로퍼티를 직접 재정의 해야 한다.             (딕셔너리를 만들어 내는 코드를 row코드라고 함. )

In [6]:
sql = "SELECT * FROM department"

try:
    cursor.execute(sql)

    # row factory 재정의 - 접속 해제 전까지 최초 1회만 수행하면 된다.
    cursor.rowfactory = lambda * args: dict(zip([d[0]for d in cursor.description], args))

    result = cursor.fetchall()
    result
except Exception as e:
    print("데이터 조회 실패", e)

### 4. 조회 결과를 활용한 후속처리

#### 수업에서 배운 내용 활용

In [7]:
with open('department.csv', 'w', encoding= 'utf-8') as f:
    for i, v in enumerate (result):
        if i == 0:
            keys = list(v.keys())
            titleLine = "%s\n" % ",".join(keys)
            f.write(titleLine)

        values = list(v.values())

        # for j, w, in enumerate(values):
        #     values[j] = str(w)
        
        # values의 모든 원소를 str 타입으로 변경
        values_str = list(map(str, values))

        

        valueLine = "%s\n" % "," .join(values_str)
        f.write(valueLine)

#### 앞으로 배울 내용 맛보기

In [23]:
df = DataFrame(result)
df


,DEPTNO,DNAME,LOC
0,101,컴퓨터공학과,1호관
1,102,멀티미디어학과,2호관
2,201,전자공학과,3호관
3,202,컴퓨터공학과,공학관
4,302,정보통신학과,6호관


In [24]:
df.to_csv("department2.csv", index=False, encoding = 'utf-8')   # csv파일 만들 때 이걸 권장하진 않음
                                                                # 반복문으로 돌리며 비동기처리하는게 더 빠

In [35]:

df.to_excel("department3.xlsx", index=False)                    # pip install openpyxl

### 5. 데이터베이스 접속 해제

cursor 객체와 dbcon 객체를 닫아서 점유하고 있는 메모리를 컴퓨터에 반납해야 한다.

자원 반납은 생성된 역순으로 진행한다.

In [9]:
try:
    cursor.close()
    dbcon.close()
except:
    print("접속상태가 아닙니다.")

# 03. sqlalchemy를 사용한 연동

데이터베이스의 종류를 구분하지 않고 공통된 연동 방법을 제공하는 인터페이스

### 1. 데이터베이스 접속하기

#### 접속 문자열 생성

`oracle+cx_oracle://계정이름:비밀번호@접속주소/SID`

In [29]:
conStr = "oracle+cx_oracle://hr:hr@localhost:1521/xe"

#### 접속

In [30]:
engine = create_engine(conStr)
dbcon = engine.connect()

### 2. 데이터 조회하기

조회 결과를 pandas의 DataFrame 객체 형태로 반환한다.

#### 특정 테이블의 모든 데이터 조회

In [31]:
df= read_sql_table('department', con=dbcon)
df

,deptno,dname,loc
0,101,컴퓨터공학과,1호관
1,102,멀티미디어학과,2호관
2,201,전자공학과,3호관
3,202,컴퓨터공학과,공학관
4,302,정보통신학과,6호관


#### 원하는 컬럼만 조회하기 

In [32]:
df = read_sql_table('student', columns=['studno', 'name', 'grade', 'idnum'], con=dbcon)
df

,studno,name,grade,idnum
0,10102,박미경,1,8405162123648
1,10103,김영균,3,8103211063421
2,10104,지은경,2,8004122298371
3,10105,임유진,2,8301212196482
4,10106,서재진,1,8511291186273
5,10107,이광훈,4,8109131276431
6,10108,류민정,2,8108192157498
7,10201,김진영,2,8206062186327
8,10202,오유석,4,7709121128379
9,10203,하나리,1,8501092378641


### 3. 데이터베이스 접속 해제  

In [33]:
dbcon.close()